## Setup

In [61]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [62]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [63]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [64]:
# Set item range table name
table = 'RPC_item_range' + datetime.today().strftime('%Y_%m_%d')

In [65]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table,
                'filter' : "AND item_sub_department_name = '400 RODENT PEST CONTROL' "})

snowflake_tools - 2025-07-08 17:01:50.906382+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...


snowflake_tools - 2025-07-08 17:01:55.966164+10:00 - Saved token with '2025-07-08 21:01:55+10:00' expiry...


## Demographic Segments (AU)

### Overall index

In [66]:
ds = con.read_sql_file_into_pandas(
    file="sql/demographic-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 RODENT PEST CONTROL'"
    }
)

In [67]:
print(f"Total sales: ${ds['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(ds['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(ds['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $11,879,605.74
Total quantity: 980,042
Total customers: 461,136


In [68]:
ds[["DEMOGRAPHIC_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,DEMOGRAPHIC_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Homeowner older kids,984296.77,0.082856,0.87
1,Homeowner younger kids,1885909.77,0.158752,0.84
2,Older homeowner,6042107.34,0.508612,1.22
3,Renter no kids,769299.70,0.064758,0.74
4,Renter with kids,721149.08,0.060705,1.00
5,Unclassifiable,103152.99,0.008683,1.27
6,Younger homeowner,1373690.09,0.115634,0.80


$12M in sales, 51% comes from Older homeowner
- Older homeowner over indexed
- Renter no kids under indexed

### Class index

In [69]:
RPC_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [70]:
summary = (
    RPC_ds[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 RODENT POISON,6590380.35,538613.0,326205,0.554764,0.549582,0.593494
7,500 RODENT REPELLENT,813756.44,22049.0,17120,0.068500,0.022498,0.031148
14,500 RODENT TRAP,4475468.95,419380.0,206310,0.376735,0.427920,0.375358


In [71]:
print(f"Total sales: ${summary['CLASS_SALES'].sum():,.2f}")
print(f"Total quantity: {int(summary['CLASS_QUANTITY'].sum()):,}")
print(f"Total customers: {int(summary['CLASS_CUSTOMERS'].sum()):,}")

Total sales: $11,879,605.74
Total quantity: 980,042
Total customers: 549,635


In [72]:
RPC_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 RODENT POISON,Homeowner older kids,511851.98,0.077667,0.937367
1,500 RODENT POISON,Homeowner younger kids,877409.75,0.133135,0.838635
2,500 RODENT POISON,Older homeowner,3843724.49,0.583233,1.146715
3,500 RODENT POISON,Renter no kids,310016.32,0.047041,0.726408
4,500 RODENT POISON,Renter with kids,280581.77,0.042574,0.701336
5,500 RODENT POISON,Unclassifiable,64670.81,0.009813,1.130103
6,500 RODENT POISON,Younger homeowner,702125.23,0.106538,0.921335
7,500 RODENT REPELLENT,Homeowner older kids,63782.39,0.078380,0.945981
8,500 RODENT REPELLENT,Homeowner younger kids,126008.45,0.154848,0.975408
9,500 RODENT REPELLENT,Older homeowner,387861.45,0.476631,0.937121


$12M sales, 55% sales comes from Poison, followed by 38% from Trap, only 7% from Repellent
- Renters under indexed in Poison, but over indexed in Repellent and Trap. 
- Homeowners with kids slightly under indexed in Poison, while Homeowners w/o kids slightly over indexed.
- Homeowners with kids slightly over indexed in Trap, while Homeowners w/o kids slightly under indexed.
- All homeowners slightly under indexed in Repellent, only renters prefer Repellent
- Younger pp prefers Traps, only older homeowner over indexed in Poison

group renters, older homeowner, rest 


Has pet


## Non-Commercial (NZ)

### Class index

In [73]:
non_commercial = con.read_sql_file_into_pandas(
    file="sql/non-commercial-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ'
    }
)

In [74]:
non_commercial

,ITEM_CLASS_NAME,TOTAL_SALES,TOTAL_QUANTITY,SALES_SHARE,QUANTITY_SHARE
0,500 RODENT POISON,1079336.62,66314.0,0.459282,0.387464
1,500 RODENT REPELLENT,217309.70,5067.0,0.092470,0.029606
2,500 RODENT TRAP,1053403.71,99768.0,0.448247,0.582931


In [75]:
print(f"Total sales: ${non_commercial['TOTAL_SALES'].sum():,.2f}")
print(f"Total quantity: {int(non_commercial['TOTAL_QUANTITY'].sum()):,}")

Total sales: $2,350,050.03
Total quantity: 171,149


$2M sales, 46% sales comes from Poison, followed by 45% from Trap, only 9% from Repellent

## Commercial Industry Segment (NZ)

### Overall index

In [76]:
cis = con.read_sql_file_into_pandas(
    file="sql/commercial-industry-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 RODENT PEST CONTROL'",
        'country' : 'NZ'
    }
)

In [77]:
print(f"Total sales: ${cis['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(cis['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(cis['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $680,769.80
Total quantity: 40,264
Total customers: 11,708


In [78]:
n = 7
top_n = cis.nlargest(n, "RANGE_SALES").copy()
top_n[["INDUSTRY_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,INDUSTRY_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Professional Computer and Scientific Services,94611.69,0.138978,1.68
1,Personal and Other Services,94416.99,0.138692,2.29
2,Rental Hiring and Real Estate Services,63417.96,0.093156,1.07
3,Manufacturing,57506.96,0.084473,1.63
4,Retail and Wholesale Trade,55575.98,0.081637,1.60
5,Residential Builder,53719.95,0.078911,0.31
6,Site Preparation and Development,35057.17,0.051496,0.73


In [79]:
print(f"Top {n} segments - sales share: {top_n['RANGE_SALES_SHARE'].sum():.2%}")
print(f"Top {n} segments - quantity share: {top_n['RANGE_QUANTITY_SHARE'].sum():.2%}")
print(f"Top {n} segments - customer share: {top_n['RANGE_CUSTOMER_SHARE'].sum():.2%}")
print(f"Top {n} segments - size share: {top_n['SEGMENT_SIZE_SHARE'].sum():.2%}")

Top 7 segments - sales share: 66.73%
Top 7 segments - quantity share: 65.61%
Top 7 segments - customer share: 59.98%
Top 7 segments - size share: 42.29%


### Class index

In [80]:
# Get data
RPC_cis = con.read_sql_file_into_pandas(
    file="sql/2-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': n, 
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [81]:
summary = (
    RPC_cis[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 RODENT POISON,206463.35,8912.0,3684,0.502429,0.374187,0.500136
7,500 RODENT REPELLENT,31260.63,687.0,305,0.076073,0.028845,0.041406
14,500 RODENT TRAP,173206.80,14218.0,3377,0.421499,0.596969,0.458458


In [82]:
print(f"Total sales: ${summary['CLASS_SALES'].sum():,.2f}")
print(f"Total quantity: {int(summary['CLASS_QUANTITY'].sum()):,}")
print(f"Total customers: {int(summary['CLASS_CUSTOMERS'].sum()):,}")

Total sales: $410,930.78
Total quantity: 23,817
Total customers: 7,366


In [83]:
RPC_cis[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 RODENT POISON,Commercial Builder,5559.59,0.026928,0.906998
1,500 RODENT POISON,Manufacturing,27341.71,0.132429,0.946305
2,500 RODENT POISON,Personal and Other Services,48389.15,0.234372,1.020055
3,500 RODENT POISON,Professional Computer and Scientific Services,50952.77,0.246788,1.071886
4,500 RODENT POISON,Rental Hiring and Real Estate Services,30808.72,0.149221,0.966912
5,500 RODENT POISON,Residential Builder,26770.53,0.129662,0.991852
6,500 RODENT POISON,Site Preparation and Development,16640.88,0.080600,0.944768
7,500 RODENT REPELLENT,Commercial Builder,950.93,0.030419,1.024607
8,500 RODENT REPELLENT,Manufacturing,5059.26,0.161841,1.156478
9,500 RODENT REPELLENT,Personal and Other Services,8123.74,0.259871,1.131037


## Drop item range table

In [84]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table}
    )

# Brand view

In [85]:
overall_brand = con.read_sql_file_into_pandas(
    file="sql/Overall_by_brand.sql",
    formatting={
        "start_date":                 start_date,
        "end_date":                   end_date,
        "sub_dept":                   "'400 RODENT PEST CONTROL'"        
    })

In [86]:
overall_brand

,CUSTOMER_TYPE_CODE,BRAND_NAME,SALES,TOTAL_TRX,TOTAL_UNITS,NUM_ITEMS_PURCHASED,OVERALL_SALES,OVERALL_TRX
0,Consumer,KIWICARE,695580.01,31730,37769.0,12,940093.19,39491
1,Consumer,BIG CHEESE,978429.28,40915,51017.0,26,1250863.29,50286
2,Consumer,CLIX,0.22,1,1.0,1,0.22,1
3,Consumer,TRAPPED!,303072.71,25195,44826.0,10,382278.67,30189
4,Commercial,KIWICARE,244513.18,7761,9740.0,12,940093.19,39491
5,Commercial,NOOSKI,5341.51,337,457.0,3,26334.23,1752
6,Consumer,NO,6960.74,365,367.0,1,8275.91,437
7,Commercial,GOTCHA,11099.77,1278,2066.0,3,63059.13,7913
8,Consumer,NOOSKI,20992.72,1415,1747.0,3,26334.23,1752
9,Commercial,BIG CHEESE,272434.01,9371,12437.0,26,1250863.29,50286
